In [ ]:
# Import needed libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import seaborn
import re
import pickle
%matplotlib inline

# 01. Scraping info from topuniversities.com

In [ ]:
qs_url = 'https://www.topuniversities.com'

## Initial postman/parsing
Trying to get the url which contains the actual data that we want to parse. Using Postman we can see that the actual ranking data which is shown on the page is generated with a request to `rank_url`, therefore it is this
link that we'll need to GET to extract all the data we're interested in.

In [ ]:
resp = requests.get(qs_url + '/university-rankings/world-university-rankings/2018')

In [ ]:
start = resp.text.find('rank_url')
start = resp.text.find('http', start)
stop = resp.text.find('.txt', start)
qs_data_url = resp.text[start:stop+len('.txt')]
print(qs_data_url)

We've still got all the escape characters, in this case backslashes, so we'll have to replace them:

In [ ]:
qs_data_url = qs_data_url.replace('\\' , '')
print(qs_data_url)

## Scraping the main data and putting everything into a `DataFrame`
Scraping everything that is contained in the `rank_url`. This is the majority of what we are interested in, the faculty and student data are contained on another page that is specific to each university. This will be scraped afterwards.
Handily enough, the data from `rank_url` is in `JSON` format, so we'll use the `JSON` parsing capabilities of 
`requests`

In [ ]:
rank_url = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt'
rank_data = requests.get(qs_data_url)
parsed_data = rank_data.json()
parsed_data.keys()

We've got a `Dict` with only one key, so let's have a look into it.

In [ ]:
print(type(parsed_data['data']))
print(len(parsed_data['data']))
parsed_data['data'][0]

We've got a list of 959 entries. Not too surprisingly the list we get in the `data` key is conveniently organised from highest to lowest ranked.

Let's now put all of this data into a single `DataFrame`. We're only interested in the top 200 universities, so we'll ignore the rest of the set.

In [ ]:
qs_df = pd.DataFrame()
for i in range(0,200):
    qs_df = qs_df.append(parsed_data['data'][i], ignore_index=True)
print(qs_df.shape)
qs_df.head()

We've now got a `DataFrame` with the main information, but we still need to scrape a page for each individual university that contains the number of total and international, faculty and students

## Scraping the specific page for each university

We will first define a handy little function to extract numbers from strings with newlines and commas, for example from `\n1,300` we want to extract only the `1300`

In [ ]:
def xtract_number(str_in):
    str_in = str_in.replace(',' , '')
    str_in = re.search(r'\d+', str_in).group()
    return str_in

Let's add the extra columns that we're going to populate

In [ ]:
columns_to_add = ['total faculty','inter faculty','total student','total inter']
qs_df = pd.concat([qs_df, pd.DataFrame(columns=columns_to_add)], axis=1)

In [ ]:
#Additional information is contained in the following tags
#<h3> Number of international students
#<h3> Number of students
#<h3> Number of academic faculty staff --> <div class="anno">In total & <div class="anno">International

The page containing the additional data we're looking for is already contained in the `url` field of our `DataFrame`. Each value we're looking for is contained in a `<div>` tag with `class=` the data we're looking for, within this tag is another `<div>` tag with `class="number"` which has the actual numeric value. We're therefore going to parse the page for each university and use `BeautifulSoup` to find all these tags. As there are several of them on each page, we'll double-check that they're  all the same.
The following step is very slow, it has to parse a lot of html for 200 entries.

In [ ]:
for idx in qs_df.index:
#for idx in range(30,200): # you can uncomment this to only parse the first few universities
    page = requests.get(qs_url + qs_df.loc[idx]['url']) # GET the page for one university
    soup = BeautifulSoup(page.text, 'html.parser') # parse it with bs4

    for column in columns_to_add:
        try:
            wrapper = soup.find_all('div', class_=column) # find the tag of interest
            if not wrapper:
                print('No data for', qs_df.loc[idx]['title'], 'concerning', column)
            values = np.zeros(len(wrapper))
            for i in range(0,len(wrapper)): # if there are several tags, we'll check they have the same values
                values[i] = xtract_number(wrapper[0].find('div', class_='number').string)
                if i>0 and values[i] != values[i-1]:
                   raise Exception('Numerical values for', qs_df.loc[idx]['title'], 'are different throughout the HTML') 
                else:
                    qs_df.loc[idx][column] = values[0]
            
        except IndexError:
            print('No data for', qs_df.loc[idx]['title'], 'concerning', column)

Furthermore, we can see that some data is missing for New York University and the Indian Institute of Science.
Going to the website and checking this by hand does indeed show that these pieces of information are missing. We'll therefore leave these as NaN to signify the missing data.

Given the time to GET and parse all this HTML, we've stored the `DataFrame` in a pickle for convenience:

In [ ]:
#pickle.dump( qs_df, open( "qs_dataframe.p", "wb" ) )
#qs_df = pickle.load( open( "qs_dataframe.p", "rb" ) )

Let's drop some of the extra columns that we don't really need, they're still in the pickle if we need them:

In [ ]:
qs_df.drop(['core_id', 'guide', 'logo', 'nid', 'url'], axis=1, inplace=True)

## Which are the best universities?

We are now going to compare the best universities in terms of ratio between faculty members: students and % of international students. Let's add these columns, they are merely operations involving the other columns

In [ ]:
qs_df['faculty:students ratio'] = qs_df['total faculty']/qs_df['total student']
qs_df['% international students'] = 100*qs_df['total inter']/qs_df['total student']

In [ ]:
qs_df.sort_values('faculty:students ratio' , ascending=False)

In [ ]:
qs_df.sort_values('% international students' , ascending=False)

## Aggregating by Country

We'll first create a new `DataFrame` which will have info aggregated by country

In [ ]:
qs_country = pd.DataFrame(columns=['Country'] + columns_to_add )
qs_country['Country'] = qs_df['country'].unique()
qs_country.set_index('Country', inplace=True)

Let's calculate the totals per country

In [ ]:
for country in qs_df['country'].unique():
    sums = qs_df[qs_df['country'] == country][columns_to_add].sum()
    qs_country.loc[country][columns_to_add] = sums

Now let's calculate the same two stats that we did per university before

In [ ]:
qs_country['faculty:students ratio'] = qs_country['total faculty']/qs_country['total student']
qs_country['% international students'] = 100*qs_country['total inter']/qs_country['total student']

In [ ]:
qs_country.sort_values('faculty:students ratio' , ascending=False).head()

In [ ]:
qs_country.sort_values('% international students' , ascending=False).head()

## Aggregating by region
Let's do the same thing but grouping per region now, we'll do this in the same way as just before.

In [ ]:
qs_region = pd.DataFrame(columns=['Region'] + columns_to_add )
qs_region['Region'] = qs_df['region'].unique()
qs_region.set_index('Region', inplace=True)
qs_region.head()

for region in qs_df['region'].unique():
    sums = qs_df[qs_df['region'] == region][columns_to_add].sum()
    qs_region.loc[region][columns_to_add] = sums
    
qs_region['faculty:students ratio'] = qs_region['total faculty']/qs_region['total student']
qs_region['% international students'] = 100*qs_region['total inter']/qs_region['total student']

In [ ]:
qs_region.sort_values('faculty:students ratio' , ascending=False)

In [ ]:
qs_region.sort_values('% international students' , ascending=False)

# 02. Scraping top 200 universities from Times Higher Education

We have the same issue as before, the HTML from the given url doesn't contain the data that we actually
want, rather it is loaded with a jQuery to a `json` somewhere else on the site. Using Postman and inspecting the html, there is only one `json` loaded on the ranking page, so we'll simply do some string handling to extract
the url of interest from the HTML.

In [ ]:
times_url = 'https://www.timeshighereducation.com/world-university-rankings/2018/world-ranking'
resp = requests.get(times_url)

In order to extract the url we want, we're first going to find where the "json" at the end of the url is located. We'll then use `rfind` to find the "http" at the beginning of this url.

In [ ]:
stop = resp.text.find('json')
start = resp.text.rfind('http', 0 , stop)
times_data_url = resp.text[start:stop+len('json')]
print(times_data_url)

As before, we need to filter out all the backlashes

In [ ]:
times_data_url = times_data_url.replace('\\' , '')
print(times_data_url)

In [ ]:
times_data = requests.get(times_data_url)
times_parsed = times_data.json()

We've got some other keys than just the `data` one, but they don't seem of use for what we're looking for. `location` is already contained in the main `data` key-value pair

In [ ]:
times_parsed.keys()

Again, it looks like `list` we get in the `data` key is conveniently organised from highest to lowest ranked:

In [ ]:
times_parsed['data'][0]

Let's create our `DataFrame` containing the top 200

In [ ]:
times_df = pd.DataFrame()
for i in range(0,200):
    times_df = times_df.append(times_parsed['data'][i], ignore_index=True)
print(times_df.shape)
times_df.head()

In [ ]:
times_df.columns

We've got a lot more information this time, let's get ride of the columns we're not interested in, after backing it up to a pickle

In [ ]:
pickle.dump( times_df, open( "times_dataframe.p", "wb" ) )
times_df = times_df[['location','name','rank','stats_student_staff_ratio','stats_number_students','stats_pc_intl_students']]

Let's now add region information based on what we have from the previous `DataFrame`

In [ ]:
times_df['Region'] = np.nan
for country in times_df['location'].unique():
    try:
        times_df.loc[times_df['location'] == country, 'Region'] = qs_df[qs_df['country'] == country]['region'].iloc[0]
    except IndexError:
        print('No region info for', country)    

In [ ]:
times_df = pickle.load( open( "times_dataframe.p", "rb" ) )
times_df = times_df[['location','name','rank','stats_student_staff_ratio','stats_number_students','stats_pc_intl_students']]

We're missing some region info about Luxembourg and the Russian Federation, so we'll add this by hand

In [ ]:
times_df.loc[times_df['location'] == 'Luxembourg', 'Region'] = 'Europe'
times_df.loc[times_df['location'] == 'Russian Federation', 'Region'] = 'Russia'

In [ ]:
times_df.head()

## Best universities

We'll change types to floats where we need it. We also need to do a little bit of string cleaning before handing it over to `Pandas`

In [ ]:
times_df.loc[:,'stats_number_students'] = times_df.loc[:,'stats_number_students'].str.replace(',' , '')
times_df.loc[:,'stats_pc_intl_students'] = times_df.loc[:,'stats_pc_intl_students'].str.replace('%' , '')

columns = ['stats_student_staff_ratio', 'stats_number_students', 'stats_pc_intl_students']
for col in columns:
    times_df.loc[:,col] = times_df.loc[:,col].astype(float)
times_df['faculty:students ratio'] = 1/times_df['stats_student_staff_ratio']

In [ ]:
times_df.sort_values('faculty:students ratio' , ascending=False).head()

In [ ]:
times_df.sort_values('stats_pc_intl_students' , ascending=False).head()

We've only got the % international students and students:staff ratio in the data from the Times, so we'll calculate the number of staff and international students from this data. Note that we don't have any info concerning % international faculty from the Times data.

In [ ]:
times_df['total inter'] = times_df['stats_number_students']*times_df['stats_pc_intl_students']/100
times_df['total inter'] = times_df['total inter'].astype(int)
times_df['total faculty'] = times_df['stats_number_students']/times_df['stats_student_staff_ratio']
times_df['total faculty'] = times_df['total faculty'].astype(int)

In [ ]:
times_df.head()

## Grouping by country

In [ ]:
columns_to_add = ['total faculty', 'stats_number_students', 'total inter']
times_country = pd.DataFrame(columns=['Country'] + columns_to_add )
times_country['Country'] = times_df['location'].unique()
times_country.set_index('Country', inplace=True)

for country in times_df['location'].unique():
    sums = times_df[times_df['location'] == country][columns_to_add].sum()
    times_country.loc[country][columns_to_add] = sums

times_country['faculty:students ratio'] = times_country['total faculty']/times_country['stats_number_students']
times_country['% international students'] = 100*times_country['total inter']/times_country['stats_number_students']
times_country.head()

In [ ]:
times_country.sort_values('faculty:students ratio' , ascending=False).head()

In [ ]:
times_country.sort_values('% international students' , ascending=False).head()

## Grouping by region

In [ ]:
columns_to_add = ['total faculty', 'stats_number_students', 'total inter']
times_region = pd.DataFrame(columns=['Region'] + columns_to_add )
times_region['Region'] = times_df['Region'].unique()
times_region.set_index('Region', inplace=True)

for region in times_df['Region'].unique():
    sums = times_df[times_df['Region'] == region][columns_to_add].sum()
    times_region.loc[region, columns_to_add] = sums
    
times_region['faculty:students ratio'] = times_region['total faculty']/times_region['stats_number_students']
times_region['% international students'] = 100*times_region['total inter']/times_region['stats_number_students']
times_region.head()

In [ ]:
times_region.sort_values('faculty:students ratio' , ascending=False)

In [ ]:
times_region.sort_values('% international students' , ascending=False)

# 03. Merging both `DataFrames`

As we've been looking at the number of students and faculty as well as their origin, we'll only keep this data in the merged `DataFrame`. 

In [ ]:
qs_df[qs_df['country'].str.contains('Russia')]

In [ ]:
qs_df['country'].unique()

In [ ]:
len(set(times_df['name'].unique()).intersection(qs_df['title'].unique()))

In [ ]:
qs_df.rename(columns={'title':'name'}, inplace=True)
qs_df.head()

In [ ]:
# Should do the merge after changing the names to get most similar
mrg_df = times_df.merge(qs_df, how='inner', on='name')
print(mrg_df.shape)
mrg_df.head()

In [ ]:
diff1 = list(set(times_df['name'].unique()).difference(qs_df['name'].unique()))
diff2 = list(set(qs_df['name'].unique()).difference(times_df['name'].unique()))


In [ ]:

for i in diff1:
    if len(qs_df[qs_df['name'].str.contains(i)]) > 0:
        qs_df.loc[qs_df['name'].str.contains(i),'name'] = i
for i in diff2:
    if len(times_df[times_df['name'].str.contains(i)]) > 0:
        times_df.loc[times_df['name'].str.contains(i),'name'] = i

In [ ]:
qs_df.loc[qs_df['name'].str.contains('Exeter'),'name'].str